In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.neighbors import NearestCentroid
# LightGBM
from lightgbm import LGBMClassifier
# xgboost
from xgboost import XGBClassifier
from loess.loess_1d import loess_1d
import os

In [2]:
def get_data(data_name, with_marker=False, norm=False, scale_factor=1e4):
    """
    description:
    for specific data_name, get the corresponding data.
    
    parameters:
    data_name: the dataset you want to get
    with_marker: whether to return marker genes
    norm: whether to normalize features(using the normalization in Seurat)
    scale_factor: size factor, default 1e4
    
    return:
    if with_marker=False, features(row:cell, col:gene, dataframe) and labels(dataframe) of raw data
    if with_marker=True, features(row:cell, col:gene, dataframe), labels(dataframe) and marker genes(array) of raw data
    """
    if data_name[:4] == 'PBMC':
        os.chdir('/home/tdeng/SingleCell/data/PBMC/integrated data')
        if data_name == 'PBMC':
            data = pd.read_hdf('PBMC_AllCells_withLables.h5', key='AllCells')
            features, labels = data.iloc[:, :-1], data.iloc[:,-1]
        elif 0 < int(data_name[4:]) < 100:
            features = pd.read_csv('raw_features_sample' + data_name[4:] + '.csv', index_col=0)
            labels = pd.read_csv('raw_labels_sample' + data_name[4:] + '.csv', usecols=[1])
        else:
            print("parameter 'data_name' is wrong!")
            return None
        
        if norm:
            features = np.log1p(features / features.sum(1).values.reshape(features.shape[0], 1) * scale_factor)
        if with_marker:
            part1 = np.squeeze(pd.read_csv('/home/tdeng/SingleCell/data/PBMC/hsPBMC_markers_10x.txt', usecols=[0]).values)
            part2 = np.squeeze(pd.read_csv('/home/tdeng/SingleCell/data/PBMC/blood_norm_marker.txt', usecols=[1]).values)
            markers = np.union1d(part1, part2)
            return features, labels, markers
        return features, labels
    elif data_name in ['muraro', 'segerstolpe', 'xin']:
        os.chdir('/home/tdeng/SingleCell/data/pancreas/separated data')
        features = pd.read_csv(data_name.capitalize() + '_pancreas_filtered.csv', index_col=0)
        labels = pd.read_csv(data_name.capitalize() + '_trueID_filtered.csv', usecols=[1])
        if norm:
            features = np.log1p(features / features.sum(1).values.reshape(features.shape[0], 1) * scale_factor)
        if with_marker:
            markers = np.squeeze(pd.read_csv('/home/tdeng/SingleCell/data/pancreas/pancreasMarkerGenes.csv',
                                             usecols=[0]).values)
            return features, labels, markers
        return features, labels
    elif data_name == 'all_pancreas':
        os.chdir('/home/tdeng/SingleCell/data/pancreas/integrated data')
        features = pd.read_csv('features.csv', index_col=0)
        labels = pd.read_csv('labels.csv', usecols=[1])
        if norm:
            features = np.log1p(features / features.sum(1).values.reshape(features.shape[0], 1) * scale_factor)
        if with_marker:
            markers = np.squeeze(pd.read_csv('/home/tdeng/SingleCell/data/pancreas/pancreasMarkerGenes.csv',
                                             usecols=[0]).values)
            return features, labels, markers
        return features, labels
    else:
        print("parameter 'data_name' is wrong!")
        return None
    
    
def get_gene_names(columns):
    """
    description:
    get gene names array from features(dataframe).
    
    parameter:
    columns:dataframe.columns
    
    return:
    an array contains gene names
    """
    if '__' in columns[0]:
        gene_names = pd.Series(columns).str.split('__',expand=True).iloc[:,0].values
    elif '\t' in columns[0]:
        gene_names = pd.Series(columns).str.split('\t', expand=True).iloc[:,1].values
    else:
        gene_names = pd.Series(columns).values
    return gene_names


def nearest_centroid_select(X, y, shrink_threshold):
    n_samples, n_features = X.shape
    le = LabelEncoder()
    y_ind = le.fit_transform(y)
    classes = le.classes_
    n_classes = classes.size
    centroids_ = np.empty((n_classes, n_features), dtype=np.float64)
    nk = np.zeros(n_classes)
    for cur_class in range(n_classes):
        center_mask = y_ind == cur_class
        nk[cur_class] = np.sum(center_mask)
    if n_classes < 2:
        raise ValueError('The number of classes has to be greater than'
                             ' one; got %d class' % (n_classes))
    if shrink_threshold is not None:
        if np.all(np.ptp(X, axis=0) == 0):
            raise ValueError("All features have zero variance. "
                                    "Division by zero.")
        dataset_centroid_ = np.mean(X, axis=0)

        # m parameter for determining deviation
        m = np.sqrt((1. / nk) - (1. / n_samples))
        # Calculate deviation using the standard deviation of centroids.
        variance = (X - centroids_[y_ind]) ** 2
        variance = variance.sum(axis=0)
        s = np.sqrt(variance / (n_samples - n_classes))
        s += np.median(s)  # To deter outliers from affecting the results.
        mm = m.reshape(len(m), 1)  # Reshape to allow broadcasting.
        ms = mm * s
        deviation = ((centroids_ - dataset_centroid_) / ms)
        # Soft thresholding: if the deviation crosses 0 during shrinking,
        # it becomes zero.
        signs = np.sign(deviation)
        deviation = (np.abs(deviation) - shrink_threshold)
        np.clip(deviation, 0, None, out=deviation)
        deviation *= signs
        return np.var(deviation, axis=0)


In [3]:

def select_features_from_importances(importances, feature_num, all_features):
    if importances.shape[0] != all_features.shape[0]:
        print('The length of importances and gene names are not the same! Please check again!')
        return None
    else:
        selected_features = []
        indices=np.argsort(importances)[::-1]
        for f in range(feature_num):
            #print ("%2d) %-*s %f" % (f+1,30,gene_name[f],importances[indices[f]]) )
            selected_features.append(all_features[indices[f]])
        return np.array(selected_features)


def evaluate_method(trusted_features, selected_features):
    """
    description:
    calculate 2 metrics: num of marker genes found and MRR(Mean Reciprocal Rank).
    """
    rank = np.argwhere(np.isin(selected_features, trusted_features))
    print('marker genes found:{}'.format(np.intersect1d(trusted_features, selected_features).shape[0]))
    print('MRR:{:.5f}'.format(np.sum(1 / (rank + 1)) / rank.shape[0]))
    

def find_not_const_index(X):
    """
    description:
    find indexes of genes which have const values.
    
    return:
    indexs of genes
    """
    mean = X.mean(axis=0)
    return np.where(mean != 0)

#calculate feature importances
def select_features_from_data(data_name, feature_num, method,all_features, X_train, y_train):
    """
    description:
    calculate the importances of features and call 'select_features_from_importances' function to get selected features.
    
    parameter:
    feature_num:the number of features you want to obtain
    method:the method you want to use
    all_features: feature names from which you select the most important ones
    X_train:expression matrix(row:cell,col:gene)
    y_train:labels
    
    return:
    an array contains selected genes
    """
    print('the method you are using is {}.'.format(method))
    if method == 'rf':  #random forest
        forest=RandomForestClassifier(n_estimators=100,n_jobs=15,random_state=0, verbose=0).fit(X_train,y_train)
        importances=forest.feature_importances_
        result = select_features_from_importances(importances, feature_num, all_features)
        return result
    elif method == 'lgb':
        lgb = LGBMClassifier(n_jobs=15, random_state=0).fit(X_train, y_train)
        importances = lgb.feature_importances_
        result = select_features_from_importances(importances, feature_num, all_features)
        return result
    elif method == 'xgb':
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        xgb = XGBClassifier(eval_metric='mlogloss', use_label_encoder=False, nthread=15).fit(X_train, y_train)
        importances = xgb.feature_importances_
        result = select_features_from_importances(importances, feature_num, all_features)
        return result
    elif method == 'seurat':
        not_const = find_not_const_index(X_train)   #(samples_mean != 0) is the same as (samples_var != 0))
        not_const_mean, not_const_var, not_const_gene = np.squeeze(X_train[:,not_const].mean(axis=0)), np.squeeze(X_train[:,not_const].var(axis=0)), all_features[not_const]
        mean_fit, var_fit, weigts = loess_1d(np.log10(not_const_mean), np.log10(not_const_var), frac=0.3, degree=2)

        z = (X_train[:,not_const] - not_const_mean) / (10 ** (var_fit / 2))
        z[z > X_train.shape[0] ** 0.5] = X_train.shape[0] ** 0.5
        z = np.var(np.squeeze(z), axis=0)
        
        result = select_features_from_importances(z, feature_num, not_const_gene)
        return result
    elif method == 'var':
        var = np.var(X_train, axis=0)
        result = select_features_from_importances(var, feature_num, all_features)
        return result
    elif method == 'cv2':
        not_const = find_not_const_index(X_train)
        print(not_const)
        not_const_X, not_const_gene = np.squeeze(X_train[:,not_const]), all_features[not_const]
        cv = np.squeeze(np.std(not_const_X, axis=0) / np.squeeze(np.mean(not_const_X, axis=0)))
        print('num of genes whose mean values are less than 1e-4:{}'.format(np.sum(np.squeeze(np.mean(not_const_X, axis=0)) < 1e-4)))
        result = select_features_from_importances(cv ** 2, feature_num, not_const_gene)
        return result
    elif method == 'nsc':
        not_const = find_not_const_index(X_train)
        not_const_X, not_const_gene = np.squeeze(X_train[:,not_const]), all_features[not_const]
        th_dict = {'shrink_threshold':np.arange(0.0, 1.01, 0.01)}
        gs = GridSearchCV(estimator=NearestCentroid(), param_grid=th_dict, cv=3, scoring='balanced_accuracy').fit(not_const_X, y_train)
        print('best score:{}, best threshold:{}'.format(gs.best_score_, gs.best_params_['shrink_threshold']))
        
        var = nearest_centroid_select(not_const_X, y_train, shrink_threshold=gs.best_params_['shrink_threshold'])
        result = select_features_from_importances(var, feature_num, not_const_gene)
        return result
    elif method == 'm3drop':
        os.chdir('/home/tdeng/R/')
        genes = pd.read_csv(data_name + '_markers.csv', usecols=[1, 3])
        genes.sort_values(by='p.value', inplace=True, ascending=True)
        return get_gene_names(genes['Gene'].values[:feature_num])# '\t' --> '.'
    else: 
        print("the parameter 'method' is wrong!")

In [4]:
def select_features(n_features):
    """
    select genes from datasets using specific methods(not split dataset).
    """
    for dataset in ['all_pancreas']:#,,'PBMC10','PBMC20','PBMC50''muraro','segerstolpe', 'xin',
        print('***************{}***************'.format(dataset))
        
        # using methods on normalized data
        X, y, trusted_markers = get_data(dataset, with_marker=True, norm=True)
        gene_names = get_gene_names(X.columns)
#         X_train, X_test, y_train, y_test = train_test_split(X.values, np.squeeze(y.values), test_size=0.3, random_state=2020, shuffle=True)
        for method in ['nsc']:#,'rf', 'lgb', 'xgb', 'var', 'cv2' 
            result = select_features_from_data(dataset, n_features, method, all_features=gene_names, X_train=X.values, y_train=np.squeeze(y.values))
            evaluate_method(trusted_markers, result)
        
        # using methods on raw data
        X, y, trusted_markers = get_data(dataset, with_marker=True, norm=False)
        gene_names = get_gene_names(X.columns)
#         X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3, random_state=2020, shuffle=True)
#         for method in ['cv2']:
#             result = select_features_from_data(n_features, method, all_features=gene_names, X_train=X.values, y_train=np.squeeze(y.values))
#             evaluate_method(trusted_markers, result)
        print('All marker genes:{}. In data:{}.'.format(trusted_markers.shape[0], np.intersect1d(gene_names, trusted_markers).shape[0]))
        
        

In [5]:
select_features(1000)

***************all_pancreas***************
the method you are using is nsc.
best score:0.7412328558287277, best threshold:1.0
marker genes found:110
MRR:0.03065
All marker genes:365. In data:312.
